In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df=pd.read_csv('combined_data.csv')
df.drop('DATE',axis=1,inplace=True)

In [49]:
X=df.drop('CSUSHPISA',axis=1).values
y=df['CSUSHPISA'].values

X_train=X[:int(len(X)*0.65)]
X_test=X[int(len(X)*0.65):]

y_train=y[:int(len(X)*0.65)]
y_test=y[int(len(X)*0.65):]

In [50]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
scaler=scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_tuner.tuners import RandomSearch
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [52]:
X_train.shape

(163, 12)

In [53]:
def build_model(hp):
    model=Sequential()
    for i in range(hp.Int('num_layers',2,5)):
        model.add(Dense(units=hp.Int('units_'+str(i),
                                     min_value=16,
                                     max_value=512,
                                     step=32),
                                     activation='elu',))
        model.add(Dropout(hp.Float('dropout', 0.1, 0.2, step=0.1, default=0.5)))
    model.add(Dense(1))
    model.compile(
        optimizer=Adam(
        hp.Choice('learning_rate',[0.0002])),
        loss='mse',
        # metrics=['mse']
    )
    return model 

In [54]:
tuner=RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=30,
    executions_per_trial=3,
    directory='hyperparam',
    project_name='parmeters'
) 

INFO:tensorflow:Reloading Oracle from existing project hyperparam/parmeters/oracle.json
INFO:tensorflow:Reloading Tuner from hyperparam/parmeters/tuner0.json


In [55]:
tuner.search(X_train, y_train,
             epochs=400,
             validation_data=(X_test, y_test),)

INFO:tensorflow:Oracle triggered exit


In [56]:
tuner.results_summary()

Results summary
Results in hyperparam/parmeters
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 4
units_0: 48
dropout: 0.2
units_1: 464
learning_rate: 0.0002
units_2: 368
units_3: 80
units_4: 16
Score: 40.34173901875814
Trial summary
Hyperparameters:
num_layers: 3
units_0: 496
dropout: 0.2
units_1: 368
learning_rate: 0.0002
units_2: 80
units_3: 48
units_4: 464
Score: 41.9729118347168
Trial summary
Hyperparameters:
num_layers: 2
units_0: 336
dropout: 0.1
units_1: 464
learning_rate: 0.0002
units_2: 144
units_3: 80
units_4: 144
Score: 43.29555892944336
Trial summary
Hyperparameters:
num_layers: 3
units_0: 48
dropout: 0.1
units_1: 336
learning_rate: 0.0002
units_2: 432
units_3: 176
units_4: 336
Score: 43.30108642578125
Trial summary
Hyperparameters:
num_layers: 4
units_0: 112
dropout: 0.2
units_1: 368
learning_rate: 0.0002
units_2: 112
units_3: 304
units_4: 304
Score: 44.39666239420573
Trial summary
Hyperparameters:
num_layers: 

In [57]:
best_hps=tuner.get_best_hyperparameters()[0]
for h_param in [f"units_{i}" for i in range(0,5)] + ['learning_rate']:
    print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units_0 48
units_1 464
units_2 368
units_3 80
units_4 16
learning_rate 0.0002


In [58]:
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=25)

In [59]:
model = tuner.hypermodel.build(best_hps)
history =model.fit(x=X_train,y=y_train,validation_data=(X_test,y_test),batch_size=32,epochs=200,callbacks=[early_stop])

Epoch 1/200
6/6 [==============================] - 1s 28ms/step - loss: 21671.4531 - val_loss: 35828.8242
Epoch 2/200
6/6 [==============================] - 0s 7ms/step - loss: 21064.7871 - val_loss: 34941.0664
Epoch 3/200
6/6 [==============================] - 0s 7ms/step - loss: 20436.7676 - val_loss: 33989.0625
Epoch 4/200
6/6 [==============================] - 0s 7ms/step - loss: 19725.5117 - val_loss: 32933.0391
Epoch 5/200
6/6 [==============================] - 0s 7ms/step - loss: 18903.4688 - val_loss: 31720.5391
Epoch 6/200
6/6 [==============================] - 0s 7ms/step - loss: 18020.8398 - val_loss: 30355.6973
Epoch 7/200
6/6 [==============================] - 0s 7ms/step - loss: 16957.7988 - val_loss: 28829.2754
Epoch 8/200
6/6 [==============================] - 0s 7ms/step - loss: 15699.3867 - val_loss: 27059.8535
Epoch 9/200
6/6 [==============================] - 0s 7ms/step - loss: 14412.0264 - val_loss: 24997.6914
Epoch 10/200
6/6 [==============================] - 0s

In [60]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 48)                624       
_________________________________________________________________
dropout (Dropout)            (None, 48)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 464)               22736     
_________________________________________________________________
dropout_1 (Dropout)          (None, 464)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 368)               171120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 368)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 80)                2

In [61]:
predictions=model.predict(X_test) 
print(mean_absolute_error(y_test,predictions))
print(np.sqrt(mean_squared_error(y_test,predictions)))

print(r2_score(y_test,predictions))

4.186065193776332
6.172330059147435
0.9087834996966879


In [65]:
# model.save('/home/l/prog/ds/Projects/home_price_pred/model')

In [63]:
# import pickle
# pickle.dump(scaler,open('scaler.pkl','wb'))